# Imports

In [1]:
import json
from pandas.io.json import json_normalize

import os
import regex as re
import string
import itertools

import numpy as np
from numpy import mean
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

import nltk
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import jiwer
from jiwer import wer

import pickle

from sklearn.ensemble import RandomForestClassifier
#nltk.download('words')
import warnings
warnings.filterwarnings("ignore")

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

In [ ]:
# own functions, placed in functions.py.
from functions import *

# Import cleaned data and prepare for alignment

In [2]:
#import data
df = pd.read_pickle("./data/df_clean_newwer.pkl")
df.shape

(298896, 10)

In [3]:
machines_keep = ['2', '3', '4', '5']
df = make_reduced_df (df, machines_keep)
df = split_hypothesis (df)

# Prepare train, validation and test data

In [4]:
# select 15% of reference texts for validation and 15% for test
all_references = list(df['reference.text'].unique()) # 18.681 reference texts in total 
reference_texts_train, reference_texts_validation, reference_texts_test = refereces_train_validation_test (all_references, 0.15, 0.15)

In [5]:
# #check for overlaps between lists
# for element in reference_texts_train:
#     matching = [s for s in reference_texts_test if element in s]
# matching

In [6]:
#Export train, validation, test lists
with open('./data/reference_texts_validation.pkl','wb') as f:
    pickle.dump(reference_texts_validation,f)
         
with open('.data./reference_texts_test.pkl','wb') as f:
    pickle.dump(reference_texts_test,f)
         
with open('./data/reference_texts_train.pkl','wb') as f:
    pickle.dump(reference_texts_train,f)
    
#Import train, validation, test lists         
# reference_texts_validation = pickle.load(open('./data/reference_texts_validation.pkl', 'rb'))
# reference_texts_test = pickle.load(open('./data/reference_texts_test.pkl', 'rb'))
# reference_texts_train = pickle.load(open('./data/reference_texts_train.pkl', 'rb'))

In [7]:
df_train_validation = df[~df['reference.text'].isin(reference_texts_test)]
df_test  = df[df['reference.text'].isin(reference_texts_test)]

In [8]:
# # checks
# df_train_validation['reference.text'].nunique() #15.879
# df_test['reference.text'].nunique() #2.802

# Alignment
On the **train and validation**, we align to REFERENCE so that we can compute accuracy. On **test** we align to the BEST configuration.

## Train and validation data - aligned to REF

In [9]:
reference_texts_train_validation = reference_texts_train + reference_texts_validation
alignment_with_ref = alignment(df_train_validation, reference_texts_train_validation, with_reference = True)

//anaconda/envs/spinning_bytes/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [15]:
alignment_with_ref.to_pickle('./data/alignment_with_ref.pkl')
#alignment_with_ref = pd.read_pickle("./data/alignment_with_ref.pkl")

## Test data - aligned to the hypotehsis of the best machine
We align **all the data** and then we separate the **test data**. The alignment of all the data is useful in order to compute in a next step the performance if majority voting. 

In [18]:
reference_texts = df['reference.text'].unique()
alignment_all = alignment(df, reference_texts, with_reference = False)
alignment_test = alignment_all[alignment_all['reference.text'].isin(reference_texts_test)]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [18]:
alignment_all.to_pickle('./data/alignment_all.pkl')
#alignment_all = pd.read_pickle("./data/alignment_all.pkl")

alignment_test.to_pickle('./data/alignment_test.pkl')
#alignment_test = pd.read_pickle("./data/alignment_test.pkl")

# Voting

In [ ]:
configurations = df['configuration'].unique()
voting_results = vote(alignment_all, reference_texts, configurations)
voting_results

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


,reference.text,voting_result,voting_wer
0,the meaning of caprivi's speech put into plain language is that funds are needed not to resist foreign foes but to buy under officers to be ready to act against the enslaved toiling masses,the meeting of the previous speech put into plain language is that funds are needed not to resist foreign foes but to buy under officers to be ready to act against the slave toiling masses,0.117647
1,one of the fellows in the outer line not brave enough himself to join the volunteers who had come in to beat us out was not disposed to believe in the surrender and had his gun levelled on bob in spite of the handkerchief which was waving as a flag of truce,one of the fellows in the outer line not brave enough himself to join the volunteers who had come in to be this out was not disposed to believe in the surrender and had his gun leveled on bob in spite of the handkerchief which was waving as a flag of truce,0.057692
2,major foster in a letter to judge george m bennett of minneapolis said,major foster in a letter to joe's georgian and of minneapolis said,0.307692
3,we took the oath perhaps three hundred of us down on luther mason's farm a few miles from where i now write where colonel hays had encamped after independence,we took the oath perhaps three hundred of us down on luther mason's farm a few miles from where i now right for colonel haze had encamped after independence,0.103448
4,foster had nearly one thousand cavalrymen and two pieces of rabb's indiana battery that had already made for itself a name for hard fighting,foster and nearly one thousand cameraman and two pieces of wraps indiana battery that had already made for itself a name for hard fighting,0.125000
...,...,...,...
18676,yeah also thinking i think both uh if we had a budget of twenty twenty uh euros it will be okay but uh,yeah i thinking i think both if we had a budget of twenty twenty euros it will be okay but,0.173913
18677,okay to take the minutes of the meeting,okay to take the minutes of the meeting,0.000000
18678,yeah rub off some of those,yet rub of some of those,0.333333
18679,okay olivier do you want to,okay but you do you want,0.500000


In [25]:
#voting_results.to_pickle('./data/voting_results_all.pkl')
#voting_results.to_csv('./data/voting_results_all.csv')
#voting_results = pd.read_pickle("./data/voting_results_all.pkl")

In [37]:
best_asr_df = make_best_asr_df(df, reference_texts)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


,reference.text,min_wer,best_config,best_hyp
0,the meaning of caprivi's speech put into plain language is that funds are needed not to resist foreign foes but to buy under officers to be ready to act against the enslaved toiling masses,0.09,"[5_ae, 5_a]","[the meaning of the previous speech put into plan language is that funds are needed not to resist foreign foes but to buy under officers to be ready to act against the enslaved toiling masses, the meaning of the previous speech put into plan language is that funds are needed not to resist foreign foes but to buy under officers to be ready to act against the enslaved toiling masses]"
1,one of the fellows in the outer line not brave enough himself to join the volunteers who had come in to beat us out was not disposed to believe in the surrender and had his gun levelled on bob in spite of the handkerchief which was waving as a flag of truce,0.02,"[5_ae, 5_a]","[one of the fellows in the outer line not brave enough himself to join the volunteers who had come in to beat us out was not disposed to believe in the surrender and had his gun leveled on bob in spite of the handkerchief which was waving as a flag of truce, one of the fellows in the outer line not brave enough himself to join the volunteers who had come in to beat us out was not disposed to believe in the surrender and had his gun leveled on bob in spite of the handkerchief which was waving as a flag of truce]"
2,major foster in a letter to judge george m bennett of minneapolis said,0.15,"[4_iaebglebg, 4_iabglbg]","[major foster in a letter to judge george invented of minneapolis said, major foster in a letter to judge george invented of minneapolis said]"
3,we took the oath perhaps three hundred of us down on luther mason's farm a few miles from where i now write where colonel hays had encamped after independence,0.07,"[5_ae, 5_a]","[we took the oath perhaps three hundred of us down on luther mason's farm a few miles from where i now right where colonel hayes had encamped after independence, we took the oath perhaps three hundred of us down on luther mason's farm a few miles from where i now right where colonel hayes had encamped after independence]"
4,foster had nearly one thousand cavalrymen and two pieces of rabb's indiana battery that had already made for itself a name for hard fighting,0.08,"[5_ae, 5_a]","[foster had nearly one thousand koevermans and two pieces of rap's indiana battery that had already made for itself a name for hard fighting, foster had nearly one thousand koevermans and two pieces of rap's indiana battery that had already made for itself a name for hard fighting]"
...,...,...,...,...
18676,yeah also thinking i think both uh if we had a budget of twenty twenty uh euros it will be okay but uh,0.22,[2_ka],[yeah i was just thinking i think both uh if we had a budget of twenty twenty euros it will be okay but um]
18677,okay to take the minutes of the meeting,0.00,"[4_iaebglebg, 4_iabglbg]","[okay to take the minutes of the meeting, okay to take the minutes of the meeting]"
18678,yeah rub off some of those,0.00,"[5_ae, 5_a]","[yeah rub off some of those, yeah rub off some of those]"
18679,okay olivier do you want to,0.50,"[5_ae, 5_a]","[okay but you do you want to think, okay but you do you want to think]"


In [22]:
assessment_voting = pd.merge(best_asr_df, voting_results, on='reference.text')
assessment_voting['voting_class'] = np.where((assessment_voting['voting_wer'] < assessment_voting['min_wer']) , 'better', np.where((assessment_voting['voting_wer'] > assessment_voting['min_wer']) , 'worse', 'same'))
#assessment_voting.to_pickle('./data/assessment_voting.pkl')
#assessment_voting.to_csv('./data/assessment_voting.csv')
#assessment_voting = pd.read_pickle("./data/assessment_voting.pkl")

In [23]:
assessment_voting

,reference.text,min_wer,best_config,best_hyp,voting_result,voting_wer,voting_class
0,the meaning of caprivi's speech put into plain...,0.09,"[5_ae, 5_a]",[the meaning of the previous speech put into p...,the meeting of the previous speech put into pl...,0.117647,worse
1,one of the fellows in the outer line not brave...,0.02,"[5_ae, 5_a]",[one of the fellows in the outer line not brav...,one of the fellows in the outer line not brave...,0.057692,worse
2,major foster in a letter to judge george m ben...,0.15,"[4_iaebglebg, 4_iabglbg]",[major foster in a letter to judge george inve...,major foster in a letter to joe's georgian and...,0.307692,worse
3,we took the oath perhaps three hundred of us d...,0.07,"[5_ae, 5_a]",[we took the oath perhaps three hundred of us ...,we took the oath perhaps three hundred of us d...,0.103448,worse
4,foster had nearly one thousand cavalrymen and ...,0.08,"[5_ae, 5_a]",[foster had nearly one thousand koevermans and...,foster and nearly one thousand cameraman and t...,0.125000,worse
...,...,...,...,...,...,...,...
18676,yeah also thinking i think both uh if we had a...,0.22,[2_ka],[yeah i was just thinking i think both uh if w...,yeah i thinking i think both if we had a budge...,0.173913,better
18677,okay to take the minutes of the meeting,0.00,"[4_iaebglebg, 4_iabglbg]","[okay to take the minutes of the meeting, okay...",okay to take the minutes of the meeting,0.000000,same
18678,yeah rub off some of those,0.00,"[5_ae, 5_a]","[yeah rub off some of those, yeah rub off some...",yet rub of some of those,0.333333,worse
18679,okay olivier do you want to,0.50,"[5_ae, 5_a]","[okay but you do you want to think, okay but y...",okay but you do you want,0.500000,same


In [54]:
stats_train = stats_voting(assessment_voting, reference_texts_train)
stats_validation = stats_voting(assessment_voting, reference_texts_validation)
stats_test = stats_voting(assessment_voting, reference_texts_test)
stats_overall = stats_voting(assessment_voting, reference_texts)

In [58]:
stats_overall

,counts,per,per100
worse,5870,0.448880,44.9%
same,5820,0.445056,44.5%
better,1387,0.106064,10.6%


In [59]:
assessment_voting.mean()
#assessment_voting[assessment_voting['reference.text'].isin(reference_texts_train)].mean()

min_wer       0.096918
voting_wer    0.142790
dtype: float64

# Run alignment and voting for one example  
Both to ref and to the best. 

### 1. Some examples to use

In [15]:
## Some examples to use
#example = ['the boy had never even noticed that there was a hole in his pouch']
#example = ['those events led to a domino effect that has yet to end']
#example = ['but they are just shutting up']
#example = ['give me precise instructions how to decontaminate the trooper']
example = ['best solution is to find an area that is predominantly sunlight or shade']
#example = ['these figures of wax look creepy']
#example = ['what difference is there between a bottle and a flagon']
#example = ['you know everyone wants to has some kind of amenities']
#example = ['governments assert that armies are needed above all for external defense but that is not true']
#example = ['actually achieving that in terms of whether the you know the power requirements of the uh such a source um you know compromise the our our need for uh you know it it being']
#example = ['governments assert that armies are needed above all for external defense but that is not true','major foster in a letter to judge george m bennett of minneapolis said']
#example = ['eventually the merchant had to hire two more employees']
#example = ['aligning all the teeth may take a year or more']
## Find more examples:
#df_train_validation['reference.text'].unique()

In [16]:
df[df['reference.text'].isin(example)]

,identifier,speaker_id,file,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer,recomputed_wer,texts
1208,PCS0_SI729,PCS0,timit__5_ae.json,timit,5_ae,5,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.153846,0.153846,"[best, solution, is, to, find, an, area, that,..."
1208,PCS0_SI729,PCS0,timit__5_a.json,timit,5_a,5,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.153846,0.153846,"[best, solution, is, to, find, an, area, that,..."
1208,PCS0_SI729,PCS0,timit__4_iaebglebg.json,timit,4_iaebglebg,4,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.230769,0.230769,"[best, solution, is, to, find, an, area, that,..."
1208,PCS0_SI729,PCS0,timit__4_iabglbg.json,timit,4_iabglbg,4,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.230769,0.230769,"[best, solution, is, to, find, an, area, that,..."
1208,PCS0_SI729,PCS0,timit__3_mdagmlg.json,timit,3_mdagmlg,3,best solution is to find an area that is predo...,best solutionist to father there that is predo...,0.384615,0.384615,"[best, solutionist, to, father, there, that, i..."
1208,PCS0_SI729,PCS0,timit__2_kl.json,timit,2_kl,2,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.153846,0.153846,"[best, solution, is, to, find, an, area, that,..."
1208,PCS0_SI729,PCS0,timit__3_mdagrmlg.json,timit,3_mdagrmlg,3,best solution is to find an area that is predo...,best solutionist to find an area that is predo...,0.153846,0.153846,"[best, solutionist, to, find, an, area, that, ..."
1208,PCS0_SI729,PCS0,timit__2_ka.json,timit,2_ka,2,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.153846,0.153846,"[best, solution, is, to, find, an, area, that,..."


### 2. Alignment to REF

In [17]:
alignment_df = alignment(df, example, with_reference = True)
alignment_df

ind,5_ae,5_a,4_iaebglebg,4_iabglbg,3_mdagmlg,2_kl,3_mdagrmlg,2_ka,REF,reference.text
0,best,best,best,best,best,best,best,best,best,best solution is to find an area that is predo...
1,solution,solution,solution,solution,solutionist,solution,solutionist,solution,solution,best solution is to find an area that is predo...
2,is,is,is,is,_,is,_,is,is,best solution is to find an area that is predo...
3,to,to,to,to,to,to,to,to,to,best solution is to find an area that is predo...
4,find,find,find,find,father,find,find,find,find,best solution is to find an area that is predo...
5,an,an,an,an,there,an,an,an,an,best solution is to find an area that is predo...
6,area,area,area,area,_,area,area,area,area,best solution is to find an area that is predo...
7,that,that,that,that,that,that,that,that,that,best solution is to find an area that is predo...
8,is,is,is,is,is,is,is,is,is,best solution is to find an area that is predo...
9,predominately,predominately,predominately,predominately,predominantly,predominantly,predominantly,predominantly,predominantly,best solution is to find an area that is predo...


### 3. Alignment to best

In [18]:
alignment_df = alignment(df, example, with_reference = False)
alignment_df

ind,5_ae,5_a,4_iaebglebg,4_iabglbg,3_mdagmlg,2_kl,3_mdagrmlg,2_ka,reference.text
0,best,best,best,best,best,best,best,best,best solution is to find an area that is predo...
1,solution,solution,solution,solution,solutionist,solution,solutionist,solution,best solution is to find an area that is predo...
2,is,is,is,is,_,is,_,is,best solution is to find an area that is predo...
3,to,to,to,to,to,to,to,to,best solution is to find an area that is predo...
4,find,find,find,find,father,find,find,find,best solution is to find an area that is predo...
5,an,an,an,an,there,an,an,an,best solution is to find an area that is predo...
6,area,area,area,area,_,area,area,area,best solution is to find an area that is predo...
7,that,that,that,that,that,that,that,that,best solution is to find an area that is predo...
8,is,is,is,is,is,is,is,is,best solution is to find an area that is predo...
9,predominately,predominately,predominately,predominately,predominantly,predominantly,predominantly,predominantly,best solution is to find an area that is predo...


### 4. voting

In [19]:
voting_results = vote(alignment_df, example, ['5_ae', '5_a', '4_iaebglebg', '4_iabglbg', '3_mdagmlg', '2_kl', '3_mdagrmlg', '2_ka'])
voting_results

,reference.text,voting_result,voting_wer
0,best solution is to find an area that is predo...,best solution is to find an area that is predo...,0.0


### 5. build_mutations function

In [29]:
ref = ['this', 'is', 'a', 'part', 'speech']
hyp = ['part', 'of', 'peech']
m, c = build_mutations(ref, hyp, -1)
m

[['DELETION', 'this', '_'],
 ['DELETION', 'is', '_'],
 ['DELETION', 'a', '_'],
 ['CORRECT', 'part', 'part'],
 ['INSERTION', '_', 'of'],
 ['SUBSTITUTION', 'speech', 'peech']]

In [30]:
c

['this', 'is', 'a', 'part', '_', 'speech']